In [2]:
%pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------- ----------------------------- 2.9/11.3 MB 18.7 MB/s eta 0:00:01
   ------------------------- -------------- 7.3/11.3 MB 19.7 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.3 MB 19.6 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 17.3 MB/s  0:00:00
   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   ------------ --------------------------- 4.2/13.1 MB 20.9 MB/s eta 0:00:01
   ------------------------- -------------- 8.4/13.1 MB 20.8 MB/s eta 0:00:01
   -------------------------------------- - 12.6/13.1 MB 20.7 MB/s eta 0:00:01
   ---------------------------------------- 13.1/13.1 MB 19.1 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (3

In [3]:
import pandas as pd

In [ ]:

# Load a CSV file
df_kiwo = pd.read_csv(r'C:\Users\giuli\Documents\Open_Campus\wetter-16\00_Data\Raw\kiwo.csv')
df_daten = pd.read_csv(r'C:\Users\giuli\Documents\Open_Campus\wetter-16\00_Data\Raw\umsatzdaten_gekuerzt.csv')
df_wetter = pd.read_csv(r'C:\Users\giuli\Documents\Open_Campus\wetter-16\00_Data\Raw\wetter.csv')


In [11]:
# merge dataframes on a common column
df_datenwetter = pd.merge(df_daten, df_wetter, on='Datum', how='left')

df_merged = pd.merge(df_datenwetter, df_kiwo, on='Datum', how='left')

In [41]:
# Display the first 5 rows
print(df_merged.head())

        id       Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0  1307011  2013-07-01            1  148.828353         6.0     17.8375   
1  1307021  2013-07-02            1  159.793757         3.0     17.3125   
2  1307031  2013-07-03            1  111.885594         7.0     21.0750   
3  1307041  2013-07-04            1  168.864941         7.0     18.8500   
4  1307051  2013-07-05            1  171.280754         5.0     19.9750   

   Windgeschwindigkeit  Wettercode  KielerWoche  Wochentag  Feiertag  
0                 15.0        20.0          0.0     Monday     False  
1                 10.0         NaN          0.0    Tuesday     False  
2                  6.0        61.0          0.0  Wednesday     False  
3                  7.0        20.0          0.0   Thursday     False  
4                 12.0         NaN          0.0     Friday     False  


In [ ]:
#save merged dataframe to a new CSV file
df_merged.to_csv(r'C:\Users\giuli\Documents\Open_Campus\wetter-16/00_Data/Processed/merged_data.csv', index=False)

In [15]:
#shape of the merged dataframe
print(df_merged.shape)

(9334, 9)


In [37]:
#descriptive statistics
print(df_merged.describe())

                 id  Warengruppe       Umsatz   Bewoelkung   Temperatur  \
count  9.334000e+03  9334.000000  9334.000000  9334.000000  9334.000000   
mean   1.559311e+06     3.088172   206.749044     4.716681    12.027564   
std    1.512503e+05     1.489002   144.545189     2.639220     7.227979   
min    1.307011e+06     1.000000     7.051201     0.000000    -8.475000   
25%    1.410123e+06     2.000000    96.897441     3.000000     6.250000   
50%    1.601102e+06     3.000000   161.900831     6.000000    11.625000   
75%    1.704223e+06     4.000000   280.644663     7.000000    17.962500   
max    1.807315e+06     6.000000  1879.461831     8.000000    31.437500   

       Windgeschwindigkeit   Wettercode  KielerWoche  
count          9334.000000  7009.000000  9334.000000  
mean             10.974930    36.900128     0.023891  
std               4.130491    27.095884     0.152718  
min               3.000000     0.000000     0.000000  
25%               8.000000    10.000000     0.000

In [29]:
#min e max dates
print("Min date:", df_merged['Datum'].min())
print("Max date:", df_merged['Datum'].max())

Min date: 2013-07-01
Max date: 2018-07-31


In [34]:
#missing values in each column
print(df_merged.isnull().sum())

id                        0
Datum                     0
Warengruppe               0
Umsatz                    0
Bewoelkung                0
Temperatur                0
Windgeschwindigkeit       0
Wettercode             2325
KielerWoche               0
Wochentag                 0
Feiertag                  0
dtype: int64


In [19]:
#fix kieler woche missing values by filling with 0
df_merged['KielerWoche'].fillna(0, inplace=True)

C:\Users\giuli\AppData\Local\Temp\ipykernel_22876\758336464.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['KielerWoche'].fillna(0, inplace=True)


In [36]:
#what days are missing Bewoelkung, Temperatur and/or     Windgeschwindigkeit
missing_weather = df_merged[df_merged[['Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']].isnull().any(axis=1)]
print(missing_weather[['Datum', 'Wochentag', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit']])

Empty DataFrame
Columns: [Datum, Wochentag, Bewoelkung, Temperatur, Windgeschwindigkeit]
Index: []


In [25]:
#insert new column day of week  
df_merged['Wochentag'] = pd.to_datetime(df_merged['Datum']).dt.day_name()
print(df_merged[['Datum', 'Wochentag']].head())


        Datum  Wochentag
0  2013-07-01     Monday
1  2013-07-02    Tuesday
2  2013-07-03  Wednesday
3  2013-07-04   Thursday
4  2013-07-05     Friday


In [32]:
#insert new column holiday (boolean) from 2013-07-01 to 2018-07-31 
holidays = [
    '2013-10-03', '2013-12-25', '2013-12-26', '2014-01-01', '2014-04-18', '2014-04-21',
    '2014-05-01', '2014-05-29', '2014-06-09', '2014-10-03', '2014-12-25', '2014-12-26',
    '2015-01-01', '2015-04-03', '2015-04-06', '2015-05-01', '2015-05-14', '2015-05-25',
    '2015-10-03', '2015-12-25', '2015-12-26', '2016-01-01', '2016-03-25', '2016-03-28',
    '2016-05-01', '2016-05-05', '2016-05-16', '2016-10-03', '2016-12-25', '2016-12-26',
    '2017-01-01', '2017-04-14', '2017-04-17', '2017-05-01', '2017-05-25', '2017-06-05',
    '2017-10-03', '2017-12-25', '2017-12-26', '2018-01-01', '2018-03-30', '2018-04-02',
    '2018-05-01', '2018-05-10', '2018-05-21', '2018-10-03', '2018-12-25', '2018-12-26'
]
df_merged['Feiertag'] = df_merged['Datum'].isin(holidays)
print(df_merged[['Datum', 'Feiertag']].head())
#save merged dataframe to a new CSV file
df_merged.to_csv(r'C:\Users\giuli\Documents\Open_Campus\wetter-16/00_Data/Processed/merged_data.csv', index=False)

        Datum  Feiertag
0  2013-07-01     False
1  2013-07-02     False
2  2013-07-03     False
3  2013-07-04     False
4  2013-07-05     False


In [33]:
#fill missing weather data with the mean of day before and after
df_merged['Bewoelkung'] = df_merged['Bewoelkung'].interpolate()
df_merged['Temperatur'] = df_merged['Temperatur'].interpolate()
df_merged['Windgeschwindigkeit'] = df_merged['Windgeschwindigkeit'].interpolate()

In [35]:
#save merged dataframe to a new CSV file
df_merged.to_csv(r'C:\Users\giuli\Documents\Open_Campus\wetter-16/00_Data/Processed/inputated_data.csv', index=False)

In [40]:
#median values before and after imputation
print("Mean Bewoelkung before imputation:", df_datenwetter['Bewoelkung'].mean())
print("Mean Bewoelkung after imputation:", df_merged['Bewoelkung'].mean())
print("Mean Temperatur before imputation:", df_datenwetter['Temperatur'].mean())
print("Mean Temperatur after imputation:", df_merged['Temperatur'].mean())
print("Mean Windgeschwindigkeit before imputation:", df_datenwetter['Windgeschwindigkeit'].mean())
print("Mean Windgeschwindigkeit after imputation:", df_merged['Windgeschwindigkeit'].mean())

Mean Bewoelkung before imputation: 4.7207469775474955
Mean Bewoelkung after imputation: 4.716680951360617
Mean Temperatur before imputation: 12.028483208199185
Mean Temperatur after imputation: 12.027563909792159
Mean Windgeschwindigkeit before imputation: 10.974779995707234
Mean Windgeschwindigkeit after imputation: 10.974930362116991
